In [1]:
#Import statements
from IPython.display import HTML, display
import hashlib as hasher

def hashbits(input):
    hash_obj = hasher.sha256()
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    hashbytes = hash_obj.digest()
    return ''.join(f'{x:08b}' for x in hashbytes)

def hash(input):
    hash_obj = hasher.sha256() #
    inputbytes = input.encode()
    #print(type(inputbytes))
    hash_obj.update(inputbytes)
    return hash_obj.hexdigest()

def numberOfInitZeros(hashStr):
  count = 0
  for i in range (0 , len(hashStr)):
    if hashStr[i] == '1':
      break
    count += 1
  return count

class Block:
    def __init__(self, data, creator=None, previous=None, nonce=0):
        self.data = data
        if previous is None:
            self.previous = None
            self.previous_hash = ""
            self.creator = Miner(0 , "0")
            self.height = 0
        else:
            self.previous = previous
            self.previous_hash = previous.hash
            self.creator = creator
            self.height = previous.height+1
        self.nonce = nonce
        self.hash = self.hash_block()
        self.children = []

    def hash_block(self):
        return hashbits(self.data+ self.creator.name + self.previous_hash + str(self.nonce))

    def print(self):
      print(self.data + " "+ self.creator.name + " " + str(self.height))
        
class Blockchain:
    def __init__(self, genesis_data, difficulty):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0

    def longestChain(self):
      max = self.chain[0]
      for block in self.chain:
        if block.height > max.height:
          max = block
      return max
        
    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        
    def print(self):
      for block in self.chain:
        block.print()
        print("________")

    def hasFork(self):
      for block1 in self.chain:
        for block2 in self.chain:
          if block1!=block2 and block1.height == block2.height:
            return True
      return False


class Miner:
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = 0
    self.name = name
    self.blockchain = blockchain
    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.longestChain()
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        self.blockchain.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1


####### Drawing blockchain, not important
def maxHeight(parent):
  if len(parent.children) == 0:
    return parent.height 
  max = 0
  for child in parent.children:
    m = maxHeight(child)
    if m> max:
      max = m
  return max
  

def drawBlockchain(parent, level, html, parentLevel, childN = 0, total = 0):
  color = "#AEF751"
  if parentLevel!=-1:
    color = "#7EDBF6"
    if type(parent.creator) is AMiner:
      color = "#F58A70"
    #elif type(parent.creator) is SelfishMiner:
    #  color = "#F59AEE"
  parent.children.sort(key=lambda x: (maxHeight(x)), reverse=True)
  xx = childN
  level += childN
  html += '<g>'
  html += '<rect x="'+str(30+ 100*parent.height)+'" y="'+str(30+ 100*level)+'" width="60" height="60" stroke="black" stroke-width="1" fill="'+color+'" />'
  html += '<text x="'+str((60+ 100*parent.height))+'" y="'+str((60+ 100*level))+'" dominant-baseline="middle" text-anchor="middle" font-family="Verdana" font-size="10" font-weight="bold" fill="black">'+str(parent.creator.name)+'</text>'
  if parentLevel != -1:
    if (parent.previous.children.index(parent)) == 0:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(95+ 100*parent.previous.height)+'" y2="'+str(60+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
    else:
      html += '<line stroke-width="1px" stroke="#000000"  x1='+str(30+ 100*parent.height)+' y1="'+str(60+ 100*level)+'" x2="'+str(65+ 100*parent.previous.height)+'" y2="'+str(95+ 100*parentLevel)+'" style="marker-end: url(#markerArrow)"/>'
  html += '</g>'
  l = level
  childN = 0
  for child in parent.children:
    html,n, t = drawBlockchain(child, l, html, level, childN, total)
    if n > 0:
      childN += n
    if t > 0:
      total += t
    l = l+1
  return html, childN+ len(parent.children)-1, total+ len(parent.children)-1


def show(bc):
  htmll = ""
  html = ""
  htmll, n, t = drawBlockchain(bc.chain[0], 0, html, -1)
  html = '<svg height="'+str(115*(n+1))+'" width="'+str(115*maxHeight(bc.chain[0]))+'">'
  html += '<defs><marker id="markerArrow" markerWidth="10" markerHeight="10" refX="2" refY="6" orient="auto"><path d="M2,2 L2,11 L10,6 L2,2" style="fill: #000000;" /> </marker> </defs>'
  html += htmll
  html += '</svg>'
  display(HTML(html))

51% Attack
51% attack happens when a miner has a mining power more than 51% of the total mining power in the system. The miner is able to grow a private chain longer than the actual longest chain.

Below is a stub for AMiner which extends Miner. Implement PoWSolver function in a way that an attacker starts from the genesis and builds its own chain. Implement attackerHasControl function to determine whether the longest chain is now under control of attacker.
Change the attacker mining power to 9 and tun the code. What is the result?
Change the attacker mining power to 10 and run the code. Try out with different locations for updateLast(). What is the impact of forks on 51% attack?
Run the code with different mining powers for attacker, and different difficulties. What is the impact of them?

In [2]:
class AMiner(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    super().__init__(miningPower, name, blockchain)
    self.lastBlock = self.blockchain.chain[0]
  
  def PoWSolver(self):
    #add this function
    for i in range(self.miningPower):
        newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
        h = newBlock.hash_block()
        count = numberOfInitZeros(h)
        if count >= self.blockchain.difficulty:
            self.blockchain.add(newBlock)
            self.lastBlock = newBlock
        self.nonce += 1

def attackerHasControl(blockchain, attacker):
  #add this function
  #return True of False
  if blockchain.longestChain().creator == attacker:
    return True
  return False

bc = Blockchain("0" , 10)
m1 = Miner(2 ,"m1", bc)
m2 = Miner(3, "m2", bc)
m3 = Miner(5, "m3", bc)
while bc.size < 20:
  m1.UpdateLast()
  m2.UpdateLast()
  m3.UpdateLast()
  m1.PoWSolver()
  m2.PoWSolver()
  m3.PoWSolver()
attacker = AMiner(11, "attacker", bc)
while not attackerHasControl(bc, attacker):
  m1.UpdateLast()
  m2.UpdateLast()
  m3.UpdateLast()
  m1.PoWSolver()
  m2.PoWSolver()
  m3.PoWSolver()
  attacker.PoWSolver()
print(bc.hasFork())
bc.print()

In [ ]:
show(bc)

In [ ]:
round = []
for i in range(20):
    bc = Blockchain("0"+str(i) , 7)
    m1 = Miner(0 ,"m1", bc)
    m2 = Miner(0, "m2", bc)
    m3 = Miner(10, "m3", bc)
    while bc.size < 10:
        m1.UpdateLast()
        m2.UpdateLast()
        m3.UpdateLast()
        m1.PoWSolver()
        m2.PoWSolver()
        m3.PoWSolver()
    attacker = AMiner(9, "attacker", bc)
    while not attackerHasControl(bc, attacker) and bc.size < 1000:
        m1.UpdateLast()
        m2.UpdateLast()
        m3.UpdateLast()
        m1.PoWSolver()
        m2.PoWSolver()
        m3.PoWSolver()
        attacker.PoWSolver()
    print(attackerHasControl(bc, attacker))
    round.append(bc.longestChain().height)
print(sum(round)/len(round))


False
False
True
False
False
True
False
True
False
False
False
True
False
False
True
False
False
False
True
False
400.4


Selfish Mining
In selfish mining, miners keep the block they found secret and start working on the next block. If a miner finds a block, they will publish the secret blocks.

Here there is a stub for SelfishMiner. There is a list privateBlocks for keeping track of blocks that miner has found but has not published them yet. Note that the function longestChain is changed to randomly return the longest block if there was 2 eqaul chains.
Add PoWSolver and UpdateLast functions. Note that a selfish miner publishes all secret blocks if he sees new blocks in the longest chain, so he will not add the new block when he creates one.
Complete checkMiner function to return the number of blocks of a miner in the longest chain.
Run the function 5/6 times. Why the results are different? What is the reason behind it?
Try out different powers for selfish miner. What is the outcome?

In [ ]:
import random

class Blockchain:
    def __init__(self, genesis_data, difficulty):
        self.chain = []
        self.chain.append(Block(genesis_data))
        self.difficulty = difficulty
        self.size = 0

    def longestChain(self):
      max = self.chain[0].height
      for block in self.chain:
        if block.height > max:
          max = block.height
      maxes = [block for block in self.chain if block.height == max]
      r = random.choices(maxes, k=1)
      return r[0]
        
    def add(self, newBlock):
        self.chain.append(newBlock)
        newBlock.previous.children.append(newBlock)
        self.size +=1
        
    def print(self):
      for block in self.chain:
        block.print()
        print("________")

    def hasFork(self):
      for block1 in self.chain:
        for block2 in self.chain:
          if block1!=block2 and block1.height == block2.height:
            return True
      return False

    def checkMiner(self, miner):
      last = self.longestChain()
      count = 0
      while last!=None:
        if last.creator == miner:
          count += 1
        last = last.previous
      return count

class Miner:
  def __init__(self, miningPower, name, blockchain=None):
    self.miningPower = miningPower
    self.nonce = random.randint(0,100000)
    self.name = name
    self.blockchain = blockchain
    self.blocks = {} #
    self.children = {} #

    if self.blockchain != None:
      self.lastBlock = blockchain.longestChain()
  
  def UpdateLast(self):
    latest = self.blockchain.longestChain()
    if latest.height > self.lastBlock.height:
        self.lastBlock = latest

  def PoWSolver(self):
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        self.blockchain.add(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

class SelfishMiner(Miner):
  def __init__(self, miningPower, name, blockchain=None):
    super().__init__(miningPower, name, blockchain)
    self.privateBlocks = []
    self.publishNext = False

  def UpdateLast(self):
    #add this function
    latest = self.blockchain.longestChain()
    publicheight = latest.height
    if publicheight > self.lastBlock.height:
        self.privateBlocks = []
        self.lastBlock = latest
        self.publishNext = False
    if publicheight == self.lastBlock.height-1 and len(self.privateBlocks)> 1:
        for block in self.privateBlocks:
            self.blockchain.add(block)
            self.privateBlocks = [] 
    if publicheight == self.lastBlock.height:
        for block in self.privateBlocks:
            self.blockchain.add(block)
            self.privateBlocks = []
            self.publishNext = True

  def PoWSolver(self):
    #add this function
    for i in range (0 , self.miningPower):
      newBlock = Block(str(self.blockchain.size), self, self.lastBlock, self.nonce)
      h = newBlock.hash_block()
      count = numberOfInitZeros(h)
      if count >= bc.difficulty:
        if self.publishNext:
            self.blockchain.add(newBlock)
            self.publishNext = False
        else:
            self.privateBlocks.append(newBlock)
        self.lastBlock = newBlock
      self.nonce += 1

def attackerHasControl(blockchain, attacker):
  if blockchain.longestChain().creator == attacker:
    return True
  return False

bc = Blockchain("0" , 5)
miners = []
for i in range(15):
  m = Miner(1 ,"m"+str(i), bc)
  miners.append(m)
""" big = Miner(5,"big",bc)
while bc.size < 301:
  big.PoWSolver()

  for m in miners: 
    m.PoWSolver()
  big.UpdateLast()
  for m in miners:
    m.UpdateLast() """

    
  
selfish = SelfishMiner(5, "selfish", bc)
while bc.size < 101:
  selfish.PoWSolver()

  for m in miners:
    m.PoWSolver()

  selfish.UpdateLast()

  for m in miners:
    m.UpdateLast()

print(bc.hasFork())
print(bc.checkMiner(selfish))
#print(bc.checkMinerAll(selfish))
print(bc.longestChain().height)
print(bc.size)
print("Fraction in chain {}".format(bc.checkMiner(selfish) /bc.longestChain().height ))
#print("Fraction lost {}".format((bc.checkMinerAll(selfish) - bc.checkMiner(selfish))/ (bc.size - bc.longestChain().height)))
total = selfish.miningPower
for m in miners:
    total += m.miningPower
print("alpha {}".format(selfish.miningPower/ total))
      
#bc.print()


True
18
68
102
Fraction in chain 0.2647058823529412
alpha 0.25


In [ ]:
show(bc)
